Shell for the functions needed for the gpt model

In [21]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
from tqdm.auto import tqdm

key = jax.random.PRNGKey(42)

# Hyperparameters
batch_size = 4
context_lenght = 8
train_test_split_size = 0.9

In [22]:
def open_data(path: str = "new_nietzsche.txt"):
    txt = open(path, "r", encoding="utf-8").read()
    return txt

text = open_data()

In [23]:
class Tokenizer:
    """
    Class that takes care of encoding and decoding the text
    """

    def __init__(self, text:str, tokenizer_type:str="base") -> None:
        self.tokenizer_type = tokenizer_type
        self.vocab_size, self.all_characters = self.sort_characters(text)


    def get_vocab_size(self):
        return jnp.copy(self.vocab_size)

    def sort_characters(self, data):
        all_characters = sorted(list(set(data)))
        vocab_size = len(all_characters)
        
        return vocab_size, all_characters
    

    def encode(self, text):
        encoded_text = []
        if self.tokenizer_type == "base":
            for c in text:
                num = self.all_characters.index(c)
                encoded_text.append(num)
        return jnp.array(encoded_text)

    def decode(self, encoded_text):
        text = []
        if self.tokenizer_type == "base":
            for n in encoded_text:
                char = self.all_characters[n]
                text.append(char)
            text = "".join([str(item) for item in text])

        return text

In [24]:
tokenizer = Tokenizer(text=text, tokenizer_type="base")
data = tokenizer.encode(text)
len(data)

3396780

In [25]:
# test tokenizer
print(tokenizer.decode(data[:500]))

What I am now going to relate is the history of the next two centuries.
I shall describe what will happen, what must necessarily happen:
the triumph of Nihilism. This history can be written already; for
necessity itself is at work in bringing it about. This future is
already proclaimed by a hundred different omens; as a destiny it
announces its advent everywhere, for this music of to-morrow all ears
are already pricked. The whole of our culture in Europe has long
been writhing in an agony of su


In [26]:
class BatchLoader:
    def __init__(self, data, train_test_split_size) -> None:
        self.training_data, self.validation_data = self.splitting_data(
            data, train_test_split_size
        )

    def splitting_data(self, data, split_size):
        n = int(split_size * len(data))
        training_data = data[:n]
        validation_data = data[n:]
        return training_data, validation_data

    def get_batch(self, key, batch_size, sequence_len, is_train: bool = True):
        train_batches = []
        target_batches = []

        if is_train:
            b_data = self.training_data
        else:
            b_data = self.validation_data

        for _ in range(batch_size):
            key, subkey = jax.random.split(key)
            pos = jax.random.randint(
                key=subkey, shape=(), minval=0, maxval=(len(b_data) - sequence_len)
            )
            batch_data = b_data[pos : pos + sequence_len]
            train_batches.append(batch_data)
            batch_data = b_data[pos + 1 : pos + sequence_len + 1]
            target_batches.append(batch_data)
            key = subkey

        train_batch = jnp.stack(train_batches)
        target_batch = jnp.stack(target_batches)

        return train_batch, target_batch

In [27]:
batch_loader = BatchLoader(data=data, train_test_split_size=train_test_split_size)
train_batch, target_batch = batch_loader.get_batch(
    key, batch_size, context_lenght, is_train=True
)
print(train_batch)  # training batch
print(target_batch) # training batch shifted forward by one

[[71 70  1 71 62  1 79 65]
 [57 76  1 76 64 61  1 79]
 [ 1 76 71  1 76 64 61  0]
 [57 74 81  1 59 71 74 71]]
[[70  1 71 62  1 79 65 75]
 [76  1 76 64 61  1 79 71]
 [76 71  1 76 64 61  0 77]
 [74 81  1 59 71 74 71 68]]


In [36]:
class BenchmarkModel(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(
            num_embeddings=self.vocab_size, features=self.vocab_size
        )

    def __call__(self, data):
        logits = self.token_embedding_table(data)
        return logits

    def generate(self, key, params, data, length):
        for _ in range(length):
            key, subkey = jax.random.split(
                key
            )  # bcs every character has to be different
            print("data: ", data.shape)
            logits = self.apply({"params": params}, data)
            print("logits: ", logits.shape)
            logits = logits[:, -1, :]
            print("logits last time step: ", logits.shape)
            probabilities = jax.nn.softmax(logits)
            print("probabilities", probabilities.shape)
            probabilities = jax.numpy.squeeze(probabilities)
            print("squeezed probabilities: ", probabilities.shape)
            next_token = jax.random.choice(
                subkey, jax.numpy.arange(self.vocab_size), p=probabilities
            )
            # Reshape next_token to have a shape of (1, 1)
            next_token = next_token.reshape((1, 1))
            data = jax.numpy.concatenate((data, next_token), axis=1)

        return data

In [37]:
# Model initialization

data = jnp.ones(
    (batch_size, context_lenght), dtype=jnp.int32
)  # Example shape (batch_size, sequence_length)
labels = jnp.ones((batch_size, context_lenght), dtype=jnp.int32)

model = BenchmarkModel(vocab_size=tokenizer.get_vocab_size())

key, subkey = jax.random.split(key)
params = model.init(rngs=subkey, data=data)

In [38]:
# Generate without training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=params["params"],
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)

data:  (1, 1)
logits:  (1, 1, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 2)
logits:  (1, 2, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 3)
logits:  (1, 3, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 4)
logits:  (1, 4, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 5)
logits:  (1, 5, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 6)
logits:  (1, 6, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 7)
logits:  (1, 7, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 8)
logits:  (1, 8, 160)
logits last time step:  (1, 160)
probabilities (1, 160)
squeezed probabilities:  (160,)
data:  (1, 9)
lo

In [39]:
# @jax.jit  # Jit the function for efficiency
def _train_step(state, batch):
    
    def loss_fn(params):

        data, labels = batch
        print(data.shape)
        # Same as model.apply
        logits = state.apply_fn(
            {"params": params},
            data,
        )

        b, t, c = logits.shape
        logits = logits.reshape((b * t, c))
        labels = labels.reshape((b * t))
        labels_one_hot = nn.one_hot(labels, num_classes=c)


        loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
        mean_loss = jnp.mean(loss)
        return mean_loss, logits

    # Gradient function
    grad_fn = jax.value_and_grad(
        loss_fn,  # Function to calculate the loss
        has_aux=True,  # Function has additional outputs, here accuracy
    )
    # Determine gradients for current model, parameters and batch
    (loss, logits), grads = grad_fn(state.params)
    # accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)

    # Perform parameter update with gradients and optimizer
    state = state.apply_gradients(grads=grads)
    # Return state and any other value we might want
    return state, loss


@jax.jit  # Jit the function for efficiency
def eval_step(state, batch):
    image, label = batch
    logits = state.apply_fn({"params": state.params}, label)
    b, t, c = logits.shape
    logits = logits.reshape((b * t, c))
    labels = labels.reshape((b * t))
    labels_one_hot = nn.one_hot(labels, num_classes=tokenizer.get_vocab_size())

    loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
    return loss


def train(state, num_epochs=100):
    
    for epoch in tqdm(range(num_epochs)):
        train_batch, labels_batch = batch_loader.get_batch(
            key, batch_size, context_lenght, is_train=True
        )

        batch = (train_batch, labels_batch)
        
        epoch_loss = jnp.array([])
        epoch_acc = jnp.array([])

        # for batch in batches:
        state, loss = _train_step(state, batch)

        jnp.append(epoch_loss, loss)
        # epoch_acc.append(acc)
        # We could use the loss and accuracy for logging here, e.g. in TensorBoard

        print(f"Loss at epoch {epoch}: {loss}")
        # print(f"Accuracy at epoch {epoch}: {epoch_acc.mean()}")
    return state

In [33]:
# Optimizer

scheduler = optax.warmup_cosine_decay_schedule(
    init_value=0.01, peak_value=1, warmup_steps=100, decay_steps=2000
)
optimizer = optax.adamw(scheduler)

In [34]:
# Training

model_state = train_state.TrainState.create(
    apply_fn=model.apply,
    params=params["params"],
    tx=optimizer,
)

trained_model_state = train(model_state, num_epochs=50)

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:00<00:05,  8.33it/s]

(4, 8)
Loss at epoch 0: 5.090754508972168
(4, 8)
Loss at epoch 1: 5.0710272789001465
(4, 8)
Loss at epoch 2: 5.031786918640137
(4, 8)
Loss at epoch 3: 4.973067283630371
(4, 8)
Loss at epoch 4: 4.894927501678467
(4, 8)
Loss at epoch 5: 4.7974534034729


 24%|██▍       | 12/50 [00:00<00:01, 34.53it/s]

(4, 8)
Loss at epoch 6: 4.680770397186279
(4, 8)
Loss at epoch 7: 4.545057773590088
(4, 8)
Loss at epoch 8: 4.390567779541016
(4, 8)
Loss at epoch 9: 4.217658996582031
(4, 8)
Loss at epoch 10: 4.02683687210083
(4, 8)
Loss at epoch 11: 3.8188209533691406
(4, 8)
Loss at epoch 12: 3.594632863998413
(4, 8)
Loss at epoch 13: 3.355722188949585
(4, 8)
Loss at epoch 14: 3.1041345596313477


...

(4, 8)
Loss at epoch 26: 0.8118927478790283
(4, 8)
Loss at epoch 27: 0.7710455656051636
(4, 8)
Loss at epoch 28: 0.7423158884048462
(4, 8)
Loss at epoch 29: 0.7222499847412109
(4, 8)
Loss at epoch 30: 0.7083274126052856
(4, 8)
Loss at epoch 31: 0.6987534761428833
(4, 8)
Loss at epoch 32: 0.6922469735145569
(4, 8)
Loss at epoch 33: 0.6878734230995178


 84%|████████▍ | 42/50 [00:01<00:00, 41.48it/s]

(4, 8)
Loss at epoch 34: 0.684934675693512
(4, 8)
Loss at epoch 35: 0.6829032897949219
(4, 8)
Loss at epoch 36: 0.681389570236206
(4, 8)
Loss at epoch 37: 0.6801245808601379
(4, 8)
Loss at epoch 38: 0.6789486408233643
(4, 8)
Loss at epoch 39: 0.6777970790863037
(4, 8)
Loss at epoch 40: 0.6766810417175293
(4, 8)
Loss at epoch 41: 0.6756598353385925


100%|██████████| 50/50 [00:01<00:00, 40.00it/s]

(4, 8)
Loss at epoch 42: 0.6748096942901611
(4, 8)
Loss at epoch 43: 0.6741922497749329
(4, 8)
Loss at epoch 44: 0.6738302111625671
(4, 8)
Loss at epoch 45: 0.6736963987350464
(4, 8)
Loss at epoch 46: 0.673719048500061
(4, 8)
Loss at epoch 47: 0.6738020181655884
(4, 8)
Loss at epoch 48: 0.6738522052764893
(4, 8)
Loss at epoch 49: 0.6738056540489197


In [35]:
# Generate after training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=trained_model_state.params,
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)

(1, 1)
(1, 1, 160)
(1, 160)
(1, 2)
(1, 2, 160)
(1, 160)
(1, 3)
(1, 3, 160)
(1, 160)
(1, 4)
(1, 4, 160)
(1, 160)
(1, 5)
(1, 5, 160)
(1, 160)
(1, 6)
(1, 6, 160)
(1, 160)
(1, 7)
(1, 7, 160)
(1, 160)
(1, 8)
(1, 8, 160)
(1, 160)
(1, 9)
(1, 9, 160)
(1, 160)
(1, 10)
(1, 10, 160)
(1, 160)
(1, 11)
(1, 11, 160)
(1, 160)
(1, 12)
(1, 12, 160)
(1, 160)
(1, 13)
(1, 13, 160)
(1, 160)
(1, 14)
(1, 14, 160)
(1, 160)
(1, 15)
(1, 15, 160)
(1, 160)
(1, 16)
(1, 16, 160)
(1, 160)
(1, 17)
(1, 17, 160)
(1, 160)
(1, 18)
(1, 18, 160)
(1, 160)
(1, 19)
(1, 19, 160)
(1, 160)
(1, 20)
(1, 20, 160)
(1, 160)
(1, 21)
(1, 21, 160)
(1, 160)
(1, 22)
(1, 22, 160)
(1, 160)
(1, 23)
(1, 23, 160)
(1, 160)
(1, 24)
(1, 24, 160)
(1, 160)
(1, 25)
(1, 25, 160)
(1, 160)
(1, 26)
(1, 26, 160)
(1, 160)
(1, 27)
(1, 27, 160)
(1, 160)
(1, 28)
(1, 28, 160)
(1, 160)
(1, 29)
(1, 29, 160)
(1, 160)
(1, 30)
(1, 30, 160)
(1, 160)
(1, 31)
(1, 31, 160)
(1, 160)
(1, 32)
(1, 32, 160)
(1, 160)
(1, 33)
(1, 33, 160)
(1, 160)
(1, 34)
(1, 34, 160)
(1, 160